# Architettura di Unity: Una Visione Completa

## Il Cuore Duale di Unity

Unity utilizza un'**architettura ibrida** che combina due mondi:

**C/C++ Nativo (Il Motore):**
- Il core engine è scritto in C/C++ nativo per massime performance
- Gestisce rendering, physics, audio, e altre operazioni critiche
- Lavora direttamente con l'hardware

**C# Wrapper (L'Interfaccia):**
- Fornisce un'interfaccia C# che "avvolge" il motore nativo
- Permette agli sviluppatori di scrivere codice in C# invece che C/C++
- Traduce le tue istruzioni C# in chiamate al motore nativo

### Analogia per Comprendere
Pensa a Unity come a un'automobile di lusso:
- Il motore (C/C++) è sotto il cofano, potente e complesso
- Il cruscotto e i controlli (C#) ti permettono di guidare senza dover essere un ingegnere meccanico

## I Due Scripting Backend

Unity offre due "traduttori" diversi per eseguire il tuo codice C#:

### 1. **Mono (JIT - Just-In-Time)**
```
Codice C# → Compilazione DURANTE l'esecuzione → Esecuzione
```
**Come funziona:** Compila il codice "al volo" quando serve
**Vantaggi:**
- Compilazione rapida durante sviluppo
- Indipendente dalla piattaforma
- L'Editor Unity usa sempre questo

**Quando usarlo:** Durante lo sviluppo, per iterazioni veloci

### 2. **IL2CPP (AOT - Ahead-Of-Time)**
```
Codice C# → Compilazione PRIMA del build → C++ → Esecuzione
```
**Come funziona:** Compila TUTTO il codice prima di creare il build finale
**Vantaggi:**
- Performance migliori su alcune piattaforme
- Supporto universale (funziona ovunque)
- Build più sicuro contro reverse engineering

**Svantaggi:**
- Build più lenti
- File eseguibili più grandi

## UnityEngine.Object: Il Ponte tra Due Mondi

Questo è uno dei concetti più importanti per capire Unity:

### La Dualità degli Object
Quando crei un `GameObject` o un `Component`:

```csharp
Camera myCam = GetComponent<Camera>();
```

**In realtà esistono DUE oggetti:**
1. **Oggetto C# Managed** (nella memoria gestita da .NET)
2. **Oggetto C++ Nativo** (nella memoria nativa di Unity)

```
[Memoria C#]              [Memoria C++ Nativa]
  Camera (C#)  <---link--->  Camera (C++ data)
  ^riferimento                ^stato reale
```

### Perché Questa Architettura?

**I dati vivono in C++:**
- Le informazioni della Camera (FOV, posizione, ecc.) sono memorizzate in C++
- Questo permette al motore di accedervi velocemente per il rendering

**L'oggetto C# è un "telecomando":**
- Il tuo codice C# manipola l'oggetto C++ attraverso il wrapper
- Quando scrivi `myCam.fieldOfView = 60`, modifichi i dati nativi

## Il Sistema di Garbage Collection

Unity usa il **Boehm Garbage Collector** con due modalità:

### Incremental GC (Consigliato - Default)
```
Frame 1: Raccoglie un po' di spazzatura
Frame 2: Raccoglie un altro po'
Frame 3: Continua...
```
**Vantaggio:** Riduce gli "stuttering" (lag improvvisi)
**Come funziona:** Distribuisce il lavoro su più frame invece di fermare tutto

### "Stop the World" GC
```
*PAUSA* → Raccoglie TUTTA la spazzatura → *RIPRENDE*
```
**Problema:** Può causare freeze visibili nel gameplay

### Analogia
**Incremental:** Pulire la casa 15 minuti al giorno
**Stop the World:** Pulire tutta la casa in un giorno, niente altro

## Gestione della Memoria: Le Trappole Comuni

### 1. Il Problema del Null "Falso"
```csharp
GameObject obj;
Destroy(obj);

// L'oggetto C++ è distrutto, ma quello C# esiste ancora!
if (obj == null) // TRUE (grazie all'overload di ==)
if (obj?.transform) // COMPORTAMENTO INASPETTATO! (?. non è overloaded)
```

**IMPORTANTE:** 
- `==` e `!=` sono overloaded per UnityEngine.Object
- `?.` e `??` NON sono overloaded → possono dare risultati sorprendenti

### 2. WeakReference Non Funziona
```csharp
// NON FARE QUESTO con UnityEngine.Object!
WeakReference<Texture2D> weakRef = new WeakReference<Texture2D>(myTexture);
```
**Perché:** L'architettura duale confonde il WeakReference system

## Async/Await: Territorio Pericoloso

**Unity NON è thread-safe** per quanto riguarda l'API principale.

```csharp
// PROBLEMATICO in Unity:
async void LoadData() {
    var data = await SomeAsyncOperation();
    transform.position = data.position; // ERRORE! Non su main thread
}
```

**Problemi:**
1. La maggior parte delle API Unity funziona SOLO sul main thread
2. `async` crea oggetti → potenziali problemi di performance
3. Le task async non si fermano quando esci dal Play Mode

**Soluzione suggerita:** Usa Coroutines invece di async/await quando possibile

---




## Definizione di .NET Profile

Un profilo definisce quali **API (Application Programming Interface)** sono disponibili nel tuo progetto. Immaginali come diversi "menù" di funzionalità:

* **.NET Standard 2.0 (Menù Essenziale):** Contiene le basi universali. È leggero e garantito su ogni piattaforma.
* **.NET 4.x (Menù Completo):** Include funzionalità avanzate e specifiche per Windows, ma è più pesante e meno compatibile.

## Evoluzione dei Profili in Unity

### Era Moderna (2018 - Presente)

Oggi la scelta si riduce a due opzioni principali all'interno delle impostazioni di progetto (**Player Settings**):

| Caratteristica | .NET Standard 2.0 (Raccomandato) | .NET 4.x / Framework |
| --- | --- | --- |
| **Compatibilità** | Universale (Android, iOS, WebGL, Console) | Limitata (Alcune API sono Windows-only) |
| **Dimensione Build** | Ottimizzata e ridotta | Significativamente più grande (+7-10 MB) |
| **Versatilità** | Ideale per progetti moderni e cross-platform | Necessario solo per librerie legacy esterne |
| **Supporto** | Totale e garantito da Microsoft/Unity | Manutenuto per compatibilità con il passato |

### Era Legacy (Deprecata)

I profili **.NET 2.0** e **.NET 2.0 Subset** sono ormai obsoleti. Venivano utilizzati per supportare hardware molto vecchio (come i primi iPhone), ma oggi non offrono alcun vantaggio e mancano di sicurezza moderna.

## Compatibilità con Librerie Esterne

Quando importi un plugin (`.dll`), il profilo del tuo progetto determina se funzionerà:

1. **Progetto in .NET Standard 2.0:** Supporta pienamente plugin compilati in .NET Standard. I plugin in .NET Framework funzionano solo se non richiamano API specifiche (come `System.Drawing`).
2. **Progetto in .NET 4.x:** Supporta quasi tutto, ma a costo di build più pesanti e possibili crash su piattaforme come WebGL che non supportano le librerie di sistema Windows.

> **Il concetto di "Limitato":** Una libreria .NET Framework può girare in un progetto .NET Standard 2.0 finché richiama solo metodi comuni a entrambi. Se tenta di accedere a funzioni desktop specifiche, il gioco genererà un errore di compilazione o un crash.

## Impatto sulle Dimensioni (Caso Studio: Android)

In un progetto semplice, la scelta del profilo influisce direttamente sul runtime incluso nell'APK:

* **Build .NET Standard 2.0:** Circa **25 MB** (Runtime .NET: 4 MB).
* **Build .NET 4.x:** Circa **33 MB** (Runtime .NET: 11 MB).

**Conclusione:** Utilizzare .NET 4.x per una singola funzione superflua aggiunge circa il 30% di peso inutile alla base del tuo gioco.

## Sicurezza: TLS 1.2

Indipendentemente dal profilo, le versioni moderne di Unity supportano il protocollo **TLS 1.2** per le connessioni HTTPS tramite `UnityWebRequest`.

* Sui sistemi operativi moderni, Unity usa lo store dei certificati del sistema.
* Sulle console o sistemi chiusi, Unity utilizza certificati "root" integrati per garantire comunicazioni sicure e crittografate.

---


## Stable scripting runtime: known limitations

## 1. Il problema della dimensione del codice

L'equazione fondamentale dello sviluppo mobile e web è: **Più API disponibili = Binary più grande = Download più lento.**

Mentre il vecchio profilo **.NET 2.0 Subset** offriva circa 8.000 API, il moderno **.NET Standard 2.0** ne mette a disposizione oltre 32.000. Questo incremento garantisce strumenti potenti (come LINQ e async/await), ma raddoppia o triplica il peso del runtime incluso nel gioco.

In una build Android, passare da .NET Standard 2.0 a .NET 4.x può aggiungere oltre 6 MB di dati puramente di sistema, un costo significativo per i mercati con connessioni limitate.

## 2. L'impatto di IL2CPP sulla dimensione

Il backend di compilazione influenza drasticamente il modo in cui queste librerie pesano sul gioco:

* **Mono (JIT):** È dinamico. Carica in memoria solo le parti di libreria utilizzate effettivamente durante l'esecuzione. Il runtime rimane snello.
* **IL2CPP (AOT):** È statico. Converte tutto il codice C# in C++ prima della build. Essendo "conservativo", tende a includere intere classi o collezioni anche se ne usi solo una minima parte, per garantire che nulla manchi al momento dell'esecuzione.

## 3. Strategie di mitigazione: Code Stripping

Per contrastare l'aumento di dimensioni causato da IL2CPP e dai profili moderni, Unity utilizza il **Code Stripping**, un processo che analizza il codice e rimuove le parti inutilizzate.

### Livelli di Managed Stripping

I livelli selezionabili nei **Player Settings** determinano l'aggressività della rimozione:

| Livello | Descrizione | Pro | Contro |
| --- | --- | --- | --- |
| **Minimal** | Rimuove solo il codice Unity palesemente inutilizzato. | Massima sicurezza. | Risparmio di spazio ridotto (10-15%). |
| **Medium** | Stripping aggressivo delle librerie .NET. | Ottimo bilanciamento (30-40% risparmio). | Può rimuovere codice necessario. |
| **High** | Rimuove tutto ciò che non ha chiamate dirette. | Build minima possibile. | Molto pericoloso; rompe quasi sempre la Reflection. |

## 4. Il rischio dello Stripping: La Reflection

Il problema principale dello stripping è che il compilatore non può "vedere" le chiamate effettuate tramite **Reflection**.

Se un sistema di salvataggio (come `JsonUtility`) cerca di accedere ai campi di una classe leggendo i loro nomi come stringhe, lo stripping potrebbe aver già rimosso quei metadati perché non ha trovato riferimenti diretti nel codice C#. Risultato: il sistema di salvataggio smetterà di funzionare silenziosamente nella build finale.

### Proteggere il codice

Esistono due strumenti principali per impedire la rimozione accidentale di codice critico:

1. **Attributo `[Preserve]`:** Si applica a classi o singoli membri. Dice esplicitamente a Unity: "Non rimuovere questo elemento, anche se sembra inutilizzato".
2. **File `link.xml`:** Un file XML inserito nel progetto che elenca intere librerie (assembly) o tipi da preservare totalmente. È essenziale per i plugin esterni o le librerie di serializzazione JSON.

---


## Stack vs Heap

### 1. Fondamenti di Memoria

* **Bit/Byte:** 8 bit = 1 byte. È l'unità base per rappresentare ogni dato.
* **RAM:** Memoria di lavoro. **Veloce** e **volatile** (si svuota senza corrente).
* **Archiviazione (SSD/HDD):** Memoria **persistente** ma **lenta**. Qui i dati "riposano".

### 2. Stack vs Heap (La RAM nel dettaglio)

| Caratteristica | **Stack (Pila)** | **Heap (Mucchio)** |
| --- | --- | --- |
| **Cosa contiene** | Variabili locali, parametri, Value Types. | Oggetti, Array, Stringhe, Reference Types. |
| **Gestione** | Automatica e immediata (**LIFO**). | Gestita dal **Garbage Collector**. |
| **Velocità** | **Altissima**. | Più lenta (richiede ricerca di spazio). |
| **Limiti** | Piccolo (rischio *StackOverflow*). | Molto grande ma costoso da pulire. |

### 3. Logica dei Riferimenti ("Frecce")

* **Value Types (`struct`, `int`, `float`):** La variabile **è** il valore stesso. Copiarla crea un duplicato indipendente.
* **Reference Types (`class`, `string`, `array`):** La variabile è solo un **indirizzo** (una freccia) che punta all'oggetto sull'Heap.
* **Condivisione:** Se passi un oggetto a una funzione, passi la freccia. Entrambi agiscono sullo **stesso oggetto** originale.

### 4. Garbage Collector (GC) e Ottimizzazione

* **Isolamento:** Quando nessuna "freccia" punta più a un oggetto sull'Heap, l'oggetto diventa spazzatura.
* **Pulizia:** Il GC elimina gli oggetti isolati per liberare spazio.
* **Il Problema:** Il GC può causare **micro-scatti** (spikes) perché mette in pausa il gioco per analizzare la memoria.
* **Best Practice:** Usa l'**Object Pooling**. Invece di distruggere e creare continuamente oggetti (es. proiettili), disattivali e riutilizzali per non dare lavoro al GC.

---



## 1. Stack vs Heap: Tipi Valore e Riferimento

La memoria si divide principalmente in due aree:

* **Value Types (Tipi Valore):** (es. `int`, `float`, `bool`, `Vector3`, `Color`). Sono piccoli, veloci da copiare e vengono memorizzati nello **Stack**. Quando una funzione finisce, questa memoria sparisce all'istante senza costi.
* **Reference Types (Tipi Riferimento):** (es. `string`, `array`, `classi`). Sono potenzialmente grandi. Il dato reale vive nell'**Heap**, mentre la variabile contiene solo un piccolo "puntatore" (l'indirizzo) a quel dato.

## 2. Il Garbage Collector (GC)

Il Garbage Collector è il "netturbino" del codice. Il suo compito è trovare gli oggetti nell'Heap che non hanno più nessuno che li punta (riferimenti persi) e liberare quello spazio.

* **Come funziona:** Unity usa il collettore *Boehm–Demers–Weiser*. Scansiona tutte le variabili attive, segna ciò che è "vivo" e considera il resto "spazzatura".
* **Il problema dei "GC Spikes":** Di base, il GC è di tipo *stop-the-world*. Ferma completamente il gioco per pulire. Se la pulizia dura 50ms, il giocatore vedrà un brusco scatto nell'immagine.

## 3. Ottimizzazione: Evitare la "Spazzatura"

Il modo migliore per gestire il GC è non dargli lavoro. Ecco gli errori comuni:

### A. Concatenazione di Stringhe

Le stringhe sono immutabili. Ogni volta che fai `stringa += "nuovo"`, Unity crea una **nuova** stringa nell'Heap e butta la vecchia.

* **Soluzione:** Usa `System.Text.StringBuilder` per operazioni complesse o aggiorna il testo della UI solo quando il valore cambia davvero (non ogni frame in `Update`).

### B. Funzioni che restituiscono Array

```csharp
float[] GetNumbers() { return new float[100]; } // Crea spazzatura ogni volta

```

* **Soluzione:** Passa l'array come parametro e modificalo all'interno. In questo modo riusi la stessa memoria.

### C. Object Pooling

Invece di distruggere (`Destroy`) e creare (`Instantiate`) continuamente proiettili o nemici, è meglio "disattivarli" e rimetterli in una lista per riutilizzarli.

## 4. Strategie di Gestione del GC

Esistono due approcci principali per gestire le pulizie:

1. **Heap piccolo, pulizie frequenti:** Ideale per giochi mobile con poca memoria. Si forza il GC regolarmente (es. ogni 30 frame) per mantenere le pulizie brevi.
2. **Heap grande, pulizie rare:** Si pre-alloca molta memoria all'inizio per evitare che l'Heap si riempia durante il gameplay. Si chiama il GC solo durante le pause (es. caricamenti o menu).
## 5. Incremental Garbage Collection

È la tecnologia moderna di Unity (attiva di default). Invece di fermare il gioco per un lungo periodo, divide il lavoro del GC in piccoli frammenti distribuiti su più frame.

* **Vantaggio:** Riduce drasticamente i picchi (spikes) mantenendo il framerate stabile.
* **Limiti:** Può essere leggermente più lento se i riferimenti tra gli oggetti cambiano troppo velocemente durante la scansione (costringendo il GC a ricominciare).

---



## Referencing additional class library assemblies

Questa parte dell'architettura riguarda come "istruire" Unity a trovare pezzi di codice (le **Assembly .dll**) che non sono inclusi automaticamente nel menù standard del progetto.

### 1. Profilo .NET Standard 2.0: "Tutto incluso"

Se usi questo profilo (quello raccomandato), **non devi fare nulla**.

* Unity include già tutto ciò che fa parte del "contratto" Standard 2.0.
* Se una funzione sembra mancare, semplicemente non esiste in questo profilo e dovresti valutare il passaggio al profilo .NET 4.x.

### 2. Profilo .NET 4.x: Referenze manuali

Il profilo 4.x è più vasto, ma per mantenere le build snelle, Unity carica solo un set predefinito di librerie (come `mscorlib.dll`, `System.dll`, `System.Xml.dll`).

Se provi a usare classi come `HttpClient` e ricevi un errore di compilazione (tipo: *"The type HttpClient is defined in an assembly that is not referenced"*), devi intervenire manualmente.

#### Il File `csc.rsp`

È lo strumento che usi per parlare direttamente al compilatore C#.

* **Cos'è:** Un file di testo da creare nella cartella **Assets**.
* **Cosa fa:** Passa argomenti extra al compilatore.
* **Esempio:** Per usare `HttpClient`, devi aggiungere questa riga nel file:
> `-r:System.Net.Http.dll`


### 3. Il pericolo del cambio profilo

Il file `csc.rsp` è **specifico per il profilo scelto**.

* **Scenario critico:** Hai creato un `csc.rsp` per referenziare una libreria di .NET 4.x. Se decidi di tornare a .NET Standard 2.0, la compilazione **fallirà**.
* **Perché?** Il compilatore cercherà la libreria indicata nel file `.rsp`, ma non la troverà perché nel profilo Standard 2.0 quella specifica DLL non esiste.

> **Regola d'oro:** Ogni volta che cambi l'API Compatibility Level nelle impostazioni di Unity, ricordati di controllare (o eliminare) il file `csc.rsp` per evitare errori "fantasma" in compilazione.

---



## Configurable Enter Play Mode

### Configurable Enter Play Mode: Iterare alla velocità della luce

In Unity, il tempo è prezioso. Il **Configurable Enter Play Mode** è una feature (situata in *Project Settings > Editor*) che ti permette di ridurre drasticamente il tempo di attesa tra quando premi "Play" e quando il gioco inizia effettivamente.

Di default, Unity vuole garantirti un ambiente "pulito", identico a quello di una Build finale. Per farlo, esegue due operazioni pesanti ogni volta che premi Play:

#### 1. Domain Reload (Ricarica del Dominio)

* **Cosa fa:** Resetta completamente lo stato dello scripting. Distrugge e ricrea l'ambiente .NET.
* **Perché:** Assicura che tutte le variabili `static` vengano resettate ai loro valori iniziali e che non ci siano "residui" di esecuzioni precedenti.
* **Costo:** Alto. Più script hai, più tempo richiede.

#### 2. Scene Reload (Ricarica della Scena)

* **Cosa fa:** Distrugge tutti i GameObject attuali e li ricarica da zero dal file della Scena.
* **Perché:** Assicura che se hai spostato un oggetto o cambiato un valore durante il Play Mode precedente (o nell'Editor), tutto torni alla posizione di partenza.
* **Costo:** Medio/Alto. Dipende dalla complessità della scena (numero di oggetti, texture, ecc.).

### 🛠 Come ottimizzare (Il "Trucco")

Unity ti permette di **disabilitare** una o entrambe queste operazioni.

Se vai in `Project Settings > Editor` e abiliti **Enter Play Mode Options**, puoi deselezionare:

1. **Reload Domain:** Unity non resetterà lo stato degli script. Il gioco parte quasi istantaneamente perché la DLL è già caricata in memoria.
2. **Reload Scene:** Unity non ricaricherà la scena. Usa quella già presente in memoria nell'Editor.

**Risultato:** Il tempo di avvio passa da *secondi* a *millisecondi*.

---


### 1. Cos'è la Serializzazione?

È il processo automatico che trasforma le strutture dati (oggetti in memoria) in un formato lineare che Unity può salvare (su disco) o trasferire.
**Dove si usa:**

* **Inspector:** Visualizza i valori dei campi.
* **Prefabs & Instantiation:** Clonare un oggetto significa serializzarlo e deserializzarlo in una nuova istanza.
* **Salvataggio:** Scene, AssetBundles.
* **Hot Reloading:** Quando modifichi uno script a Editor aperto, Unity serializza tutto, ricarica le DLL, e ripristina i valori.

### 2. Le Regole d'Oro (Cosa viene serializzato)

Affinché un campo sia serializzato da Unity, deve rispettare **tutti** questi requisiti:

* Deve essere `public` OPPURE avere l'attributo `[SerializeField]`.
* NON deve essere `static`, `const`, o `readonly`.
* Il tipo di dato deve essere supportato.

**Tipi Supportati:**

* Primitivi (`int`, `float`, `bool`, `string`).
* Tipi Unity (`Vector3`, `Color`, `Quaternion`, ecc.).
* Enum.
* Array e `List<T>` (ma **niente** array multidimensionali o `Dictionary`).
* Classi e Struct custom con l'attributo `[System.Serializable]`.

### 3. Le "Trappole" del Sistema (Attenzione!) ⚠️

Come mentore, devo evidenziare questi comportamenti perché causano bug frequenti:

* **Inspector vs Properties:** Quando modifichi un valore nell'Inspector, Unity scrive direttamente nel campo serializzato. I tuoi `get` e `set` (proprietà) **vengono ignorati**.
* **Classi Custom = Struct:** Le classi custom serializzabili vengono trattate come "valori", non riferimenti. Se due script puntano alla stessa istanza di una classe custom, dopo la serializzazione avrai due copie distinte.
* **Niente Null:** Il serializzatore non supporta `null` per le classi custom inline. Crea automaticamente un'istanza vuota. Questo può causare **loop infiniti** (limite profondità: 7) se hai classi nidificate ricorsivamente.
* **Niente Polimorfismo:** Se hai una lista `List<Animale>` e ci metti un `Cane` e un `Gatto`, dopo la serializzazione torneranno a essere semplici `Animale`. I dati specifici delle sottoclassi vengono persi (a meno che non siano `ScriptableObject` o `MonoBehaviour`).

### 4. Hot Reloading (Modifica script live)

Quando salvi uno script con Unity aperto:

1. Unity serializza **tutte** le variabili (anche quelle private private senza `SerializeField`!).
2. Ricarica il dominio.
3. Ripristina i valori.

* **Nota:** Le variabili `static` vengono resettate e perse. Se vuoi che una variabile privata *non* venga ripristinata, usa `[NonSerialized]`.

### 5. Custom Serialization (`ISerializationCallbackReceiver`)

Quando le regole di Unity ti stanno strette (es. vuoi serializzare un Albero, un Dizionario o gestire il Null), usi questa interfaccia.
Ha due metodi:

* `OnBeforeSerialize()`: Trasformi i tuoi dati complessi in formati semplici (es. appiattisci un albero in una `List`).
* `OnAfterDeserialize()`: Ricostruisci i dati complessi partendo dalla lista piatta.

### 6. JSON Serialization (`JsonUtility`)

Unity ha un tool interno performante per il JSON (`JsonUtility`).

* **Pro:** Molto più veloce delle librerie .NET standard; genera zero garbage (allocazioni) se usato bene.
* **Contro:** Rigido. Non supporta Dictionaries o array "nudi" (devi wrapparli in una classe).
* **Feature Killer:** `FromJsonOverwrite(json, oggettoEsistente)`. Aggiorna i valori di un oggetto esistente senza crearne uno nuovo (ottimo per le performance).

### 7. Errori Comuni

* Mai chiamare API di Unity (es. `GameObject.Find`, `GetComponent`) dentro il **costruttore** o gli **inizializzatori di campo**.
* Fallo sempre in `Awake()` o `Start()`, o deserializzazione e costruttori andranno in conflitto.

---


## Script compilation

### 1. Le Cartelle Speciali e l'Ordine di Compilazione 

Unity non compila tutto insieme in un unico "calderone". Compila a **fasi** (Phases).
Immagina la compilazione come la costruzione di un edificio a piani: non puoi costruire il tetto se non hai finito le fondamenta.

Le cartelle che usi determinano in quale "piano" finisce il tuo script. Chi sta sopra può vedere chi sta sotto, ma **chi sta sotto non sa che esiste chi sta sopra**.

| Fase | Nome Assembly (Il "File" prodotto) | Contenuto (Cartelle) | Chi può vedere? |
| --- | --- | --- | --- |
| **1. Fondamenta** | `Assembly-CSharp-firstpass` | `Plugins`, `Standard Assets` | Nessuno (sono la base). |
| **2. Editor delle Fondamenta** | `Assembly-CSharp-Editor-firstpass` | Cartelle `Editor` dentro `Plugins` o `Standard Assets` | Vede la Fase 1. |
| **3. Il Tuo Gioco (Main)** | `Assembly-CSharp` | **Tutti gli altri script** (Il 90% del tuo codice va qui). | Vede la Fase 1. **NON vede la Fase 2 o 4.** |
| **4. I Tuoi Tool Editor** | `Assembly-CSharp-Editor` | Cartelle chiamate `Editor` nel resto del progetto. | Vede Fase 1 e 3. |

**⚠️ La Regola d'Oro:**
Uno script di gioco (Fase 3) **NON può** accedere al codice di uno script Editor (Fase 4).
Se provi a usare una classe Editor nel tuo codice di gioco, funzionerà mentre lavori in Unity, ma **la Build fallirà** perché l'Editor non esiste nel gioco finale.

### 2. Platform Dependent Compilation (Il Preprocessore) 

A volte devi scrivere codice che deve esistere solo in certe condizioni (es. solo su Android, o solo nell'Editor).
Unity usa le direttive del preprocessore C# (`#if`, `#endif`) per "tagliare via" il codice prima che il compilatore lo veda.

**Le direttive più usate:**

* `UNITY_EDITOR`: Il codice dentro esiste solo nell'Editor. Vitale per creare tool di debug che non rompano la build finale.
* `UNITY_ANDROID`, `UNITY_IOS`: Codice specifico per mobile (es. controlli touch o permessi).
* `UNITY_STANDALONE`: Windows, Mac o Linux.

**Esempio Pratico (Evitare crash in Build):**

```csharp
using UnityEngine;
// using UnityEditor; // ❌ ERRORE! Questo namespace non esiste nella Build.

public class GameController : MonoBehaviour {
    void Update() {
        if (Input.GetKeyDown(KeyCode.Space)) {
            Debug.Log("Salto!");
            
            // Questo blocco viene letto dal compilatore SOLO se siamo nell'Editor
            #if UNITY_EDITOR
                // Codice che usa UnityEditor (sicuro qui dentro)
                Debug.Log("Messaggio solo per lo sviluppatore.");
            #endif
        }
    }
}

```

### 3. Custom Defines (Le tue regole) 

Puoi creare le tue "etichette" personali per attivare o disattivare parti di codice.
Esistono due modi per farlo:

1. **Player Settings (Consigliato):**
Vai su `Project Settings > Player > Other Settings > Scripting Define Symbols`.
Scrivi qui i tuoi simboli, separati da `;`.
*Esempio:* `DEV_MODE;STEAM_BUILD`
2. **File .rsp (Avanzato):**
Crei un file di testo (es. `csc.rsp`) nella cartella Assets. Utile se vuoi definire simboli globalmente senza toccare l'editor, ma richiede ricompilazione manuale.

**Uso nel codice:**

```csharp
#if DEV_MODE
    // Questo codice viene compilato solo se ho aggiunto "DEV_MODE" nei settings
    CheatManager.UnlockAllLevels();
#endif

```

### 4. Analisi da Mentore: Assembly Definition Files (Asmdef) 

Il testo menziona brevemente gli **Assembly Definition Files**. Questo è il collegamento con la lezione precedente sulla velocità di compilazione.

Di default, Unity mette *tutti* i tuoi script (Fase 3) in un unico gigantesco file (`Assembly-CSharp.dll`). Se cambi una virgola in uno script, Unity deve ricompilare **tutto** quel file.

Usando gli **Asmdef**, puoi spezzare il tuo progetto in "moduli" (es. `Code.Gameplay`, `Code.UI`, `Code.Physics`).

* **Vantaggio 1 (Velocità):** Se modifichi la UI, Unity ricompila solo la UI, non la Fisica.
* **Vantaggio 2 (Dipendenze):** Puoi forzare regole rigide (es. "La UI non deve mai parlare direttamente con la Fisica").

---


## Assembly

### Cos'è un Assembly Definition?

Di default, Unity compila quasi tutti i tuoi script in un unico, gigantesco file chiamato `Assembly-CSharp.dll`.
Gli **Assembly Definitions** ti permettono di spezzare questo "monolite" in librerie (Assembly) più piccole e gestibili.

Immagina il tuo progetto come una libreria fisica:

* **Senza Asmdef:** Tutti i libri sono ammucchiati in una stanza enorme. Per trovarne o modificarne uno, devi spostare l'intera pila.
* **Con Asmdef:** Organizzi i libri in scaffali separati (Gameplay, UI, Core). Se modifichi un libro nello scaffale "UI", non devi toccare lo scaffale "Gameplay".

### erché usarli? (I 3 Vantaggi Chiave)

1. **Velocità di Compilazione (Iterazione Rapida):**
Quando modifichi uno script in un Assembly, Unity ricompila *solo* quell'Assembly (e quelli che dipendono da lui). Gli altri rimangono intatti. In progetti grandi, questo riduce i tempi di attesa da minuti a secondi.
2. **Modularity & Spaghetti Code Prevention:**
Gli Assembly hanno confini rigidi. Lo script A nell'Assembly "Core" non può vedere lo script B nell'Assembly "Gameplay" a meno che tu non crei un **riferimento esplicito**. Questo impedisce riferimenti circolari e codice "spaghetti".
3. **Cross-Platform & Dipendenze:**
Puoi dire a un Assembly di esistere solo su certe piattaforme (es. solo Android) o solo se un certo pacchetto è installato (es. Unity Timeline).

### Come Funzionano: Concetti Tecnici

#### 1. Creazione e Gerarchia

Basta creare un asset **Assembly Definition** (`.asmdef`) in una cartella.

* Tutti gli script in quella cartella (e sottocartelle) apparterranno a quel nuovo Assembly.
* **Eccezione:** Se una sottocartella ha un suo `.asmdef` o `.asmref`, quella parte viene staccata e diventa indipendente.

#### 2. Riferimenti (References)

Se l'Assembly A vuole usare una classe dell'Assembly B, devi andare nell'Inspector di A e aggiungere B alla lista **Assembly Definition References**.

* **Auto Referenced:** Di default, gli Assembly speciali di Unity (come il vecchio `Assembly-CSharp`) vedono i tuoi Assembly custom. Se disabiliti questa opzione, il tuo Assembly diventa "invisibile" al sistema default.

#### 3. Il Nemico: Riferimenti Circolari 

Unity vieta tassativamente i cicli:

* A dipende da B.
* B dipende da A.
* **Risultato:** Errore di compilazione. Devi ristrutturare il codice o creare un terzo Assembly "Shared" usato da entrambi.

#### 4. Le Cartelle Speciali ("Editor") ⚠️

Normalmente, gli script in una cartella chiamata `Editor` finiscono magicamente in un assembly separato che non viene incluso nella Build finale.
**Attenzione:** Se crei un `.asmdef` in una cartella che contiene una sottocartella `Editor`, quella magia si rompe. Devi creare un secondo `.asmdef` specifico dentro la cartella `Editor` e marcarlo come **Platform: Editor Only**.

### Feature Avanzate

* **Assembly Definition Reference (.asmref):** Ti permette di aggiungere file di una cartella X a un Assembly definito nella cartella Y, senza spostare fisicamente i file.
* **Version Defines:** Puoi usare simboli preprocessore (`#if USE_TIMELINE`) che si attivano automaticamente solo se nel progetto è presente una specifica versione di un pacchetto (es. Timeline 1.3+).
* **No Engine References:** Puoi creare Assembly puri C# che non dipendono da `UnityEngine`. Utile per logica condivisa con server backend .NET.

---


## Riassunto

# L'Architettura di Unity: La Guida Definitiva

Questa guida esplora come Unity "pensa", come gestisce la memoria e come compila il tuo codice. Capire questi concetti è ciò che distingue un principiante da un professionista.

## 1. Il Cuore del Motore: Architettura Ibrida

Unity non è un blocco unico, ma una fusione di due mondi che dialogano costantemente.

### C++ Nativo vs C# Managed

* **Il Motore (C++):** È la parte "sotto il cofano". Gestisce grafica, fisica e audio. È velocissimo ma complesso.
* **Lo Scripting (C#):** È il tuo volante. Unity "wrappa" (avvolge) il codice C++ in un'interfaccia C# facile da usare (`MonoBehaviour`).

### La Dualità di `UnityEngine.Object`

Quando crei un oggetto (es. una `Camera` o una `Texture2D`), ne esistono **due copie**:

1. **Wrapper C#:** Vive nella memoria gestita (.NET). È leggero, un semplice "telecomando".
2. **Oggetto Nativo C++:** Vive nella memoria nativa. Contiene i dati pesanti (pixel, vertex data).

> **La Trappola del Null:** Quando fai `Destroy(obj)`, distruggi la parte C++. La parte C# rimane in memoria finché il Garbage Collector non passa. Unity sovraccarica l'operatore `==` per farti credere che sia `null`, ma l'operatore `?.` (null check moderno) non lo sa e può causare bug.

## 2. Esecuzione del Codice: Backend e Profili

Come fa il tuo codice C# a girare su Android, PS5 e WebGL? Attraverso due possibili **Backend di Scripting**.

| Caratteristica | **Mono (JIT)** | **IL2CPP (AOT)** |
| --- | --- | --- |
| **Compilazione** | **Just-In-Time:** Compila mentre giochi. | **Ahead-Of-Time:** Compila tutto in C++ prima della build. |
| **Utilizzo** | Sviluppo (Editor), iterazione rapida. | Release finale (Store), performance alte. |
| **Pro/Contro** | Build veloci, runtime più lento. | Build lentissime, runtime veloce e sicuro. |

### Code Stripping (Il prezzo di IL2CPP)

Per ridurre le dimensioni, IL2CPP rimuove il codice che sembra inutilizzato.

* **Rischio:** Se usi la *Reflection* o la serializzazione JSON, IL2CPP potrebbe cancellare classi che ti servono.
* **Soluzione:** Usa l'attributo `[Preserve]` o un file `link.xml`.

## 3. Gestione della Memoria: Stack, Heap e GC

Capire dove finiscono i tuoi dati è vitale per evitare i lag (stuttering).

### Stack vs Heap

* **Stack (Veloce/Temporaneo):** Per *Value Types* (`int`, `struct`, `Vector3`). Si pulisce da solo quando la funzione finisce.
* **Heap (Lento/Persistente):** Per *Reference Types* (`class`, `string`, `List`). Richiede gestione manuale o automatica.

### Il Garbage Collector (GC)

È il netturbino automatico dell'Heap. Unity usa un **Incremental GC** (pulisce un po' alla volta su più frame) per evitare blocchi evidenti.

**Best Practices per non stressare il GC:**

1. **Niente Stringhe in Update:** `text.text = "Score: " + score` crea una nuova stringa (spazzatura) ogni frame.
2. **Object Pooling:** Non distruggere/creare proiettili. Disattivali e riusali.
3. **Struct invece di Classi:** Per dati semplici e temporanei, usa `struct` (va nello Stack).

## 4. Serializzazione: Salvare e Mostrare Dati

La serializzazione è il processo di trasformare un oggetto in un flusso di dati (JSON, binario, YAML). Unity la usa ovunque: Inspector, Prefab, Salvataggi.

### Regole per essere Serializzati

Affinché Unity salvi una variabile, deve essere:

* `public` o `[SerializeField]`
* NON `static`, `const`, o `readonly`

### Le Trappole della Serializzazione

1. **No Polimorfismo:** Una lista di `Animale` contenente `Cane` e `Gatto` verrà salvata come una lista di `Animale` generici. I dati specifici si perdono (a meno che non usi `ScriptableObject`).
2. **Hot Reload:** Modificare uno script a gioco avviato forza una serializzazione completa. Le variabili `static` vengono perse (resettate).

## 5. Compilazione e Architettura del Codice

Unity compila il codice a fasi (come costruire una casa: prima le fondamenta, poi il tetto).

### Ordine di Compilazione (Chi vede chi?)

1. **Plugins/Standard Assets:** Le fondamenta. Visibili da tutti.
2. **Main Scripts (Assembly-CSharp):** Il 90% del tuo gioco. Vede i Plugins.
3. **Editor Scripts:** I tool per sviluppatori. Vedono tutto il resto.
* **Regola D'Oro:** Il codice di Gioco non può MAI chiamare codice Editor (`UnityEditor`). Romperà la build.

### Assembly Definitions (.asmdef)

Per progetti grandi, spezza il "monolite" `Assembly-CSharp.dll` in piccoli pezzi modulari.

* **Vantaggio:** Se modifichi la UI, ricompila solo la UI (secondi invece di minuti).
* **Rischio:** Riferimenti circolari (A dipende da B che dipende da A) sono vietati.

---

## 6. Workflow Optimization: Configurable Enter Play Mode

Di default, quando premi "Play", Unity ricarica tutto per simulare un avvio pulito. È lento.

Puoi velocizzarlo in `Project Settings > Editor`:

1. **Disable Domain Reload:** Non resetta lo stato degli script.
* *Pericolo:* Le variabili `static` mantengono i valori vecchi. Devi resettarle a mano con `[RuntimeInitializeOnLoadMethod]`.


2. **Disable Scene Reload:** Non ricarica la scena dal disco.
* *Sicuro:* Generalmente non crea problemi e fa risparmiare molto tempo.

---
